---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="6e93f194" -->
# Table des matières
1. [Préparation des données](#préparation-des-données)
1. [Entraînement d'un régresseur par forêt aléatoire](#entraînement-dun-régresseur-par-forêt-aléatoire)
1. [Sélection du type de régresseur](#sélection-du-type-de-régresseur)
1. [Affichage des résultats](#affichage-des-résultats)
1. [Identification des variables les plus importantes déterminant le prix d'une maison](#identification-des-variables-les-plus-importantes-déterminant-le-prix-dune-maison)
1. [Affichage du diagramme d'importance des variables du jeu de données](#affichage-du-diagramme-dimportance-des-variables-du-jeu-de-données)
1. [Comment varie le prix des maisons en fonction des deux variables les plus importantes?](#comment-varie-le-prix-des-maisons-en-fonction-des-deux-variables-les-plus-importantes)
1. [Sélection des variables d'intérêt](#sélection-des-variables-dintérêt)
1. [Affichage de l'effet de chaque variable sur le prix des maisons](#affichage-de-leffet-de-chaque-variable-sur-le-prix-des-maisons)

# Attention!
Ne lancez pas l'exécution automatique du notebook en entier en cliquant sur le bouton **Tout exécuter**. L'exécution serait interrompue, car certaines cellules exigent une entrée de votre part!

Il faut simplement exécuter le notebook, une cellule à la fois, et entrer quelques lignes de code lorsque demandées. Il est inutile de sauter ces cellules pour aller aux suivantes car celles-ci ont justement besoin de votre input!

Importons d'abord les librairies nécessaires.
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sns.set(color_codes=True)


seed = 42
np.random.seed(seed)



<!-- #region id="f2ea64bc" -->
Dans cet exemple, nous allons appliquer la régression par forêt aléatoire afin de prédire le prix, dans
les années 70, des maisons dans la région de Boston au Massachusetts (États-Unis).

Le  jeu de données contient 13 variables. Les données sont toutes numériques. Voici la liste des facteurs mesurés. Le but est de déterminer comment varie le prix des maisons en fonction de ces facteurs:

- `CRIM`:  taux de criminalité.
- `ZN`:  proportion de terrain résidentiel zoné pour les lots de plus de 25 000 pi2.
- `INDUS`:  proportion d'acres commerciales non commerciales par ville.
- `CHAS`:  indicateur de proximité avec la Charles River (= 1 si près la rivière ; 0 sinon).
- `NOX`:  concentration d'oxydes nitriques (parties par 10 millions).
- `RM`:  nombre moyen de pièces par logement.
- `AGE`:  proportion d'unités construites avant 1940 occupées par leur propriétaire.
- `DIS`:  distances pondérées à cinq centres d'emploi de Boston.
- `RAD`:  indice d'accessibilité aux autoroutes radiales.
- `TAX`:  taux d'impôt foncier sur la valeur totale par tranche de ~\$10 000.
- `PTRATIO`:  ratio élèves enseignants par ville.
- `B`:  $1000(Bk - 0.63)^2$ où $Bk$ est la proportion de personnes d'ascendance afro-américaine.
- `LSTAT`:  pourcentage des classes défavorisées dans la population.
- `MEDV`:  valeur médiane des logements occupés par leur propriétaire en k~\$. 

> À noter que ces variables ont été collectées dans les années 1970. Une variable telle que « B » serait aujourd'hui considérée comme discriminatoire et ne serait plus collectée.
<!-- #endregion -->

<!-- #region id="72520c76" -->
C'est un excellent exemple de régression basée sur des données hétéroclites qui varient de façon discontinue.

Comme on l'a vu dans les précédentes sections, la régression sert à prédire une réponse $y$, qui est
une variable numérique continue, en fonction de plusieurs facteurs/variables $x_{i}$. Soit l'équation suivante

$$y=f(x_{1}, \cdots, x_{N}, \Theta)$$

où $\Theta$ représente l'ensemble des paramètres de la fonction $f$. Dans plusieurs domaines, comme en physique, on peut
dériver analytiquement une telle formule basée sur les lois de la nature. Dans les sciences de la vie et les sciences
sociales, c'est généralement impossible dû au grand nombre de phénomènes impliqués lors de la prise de données. La
modélisation par forêt aléatoire permet de se libérer de cette contrainte irréaliste en modélisation analytique. De
par sa nature, la forêt aléatoire permet d'explorer naturellement un très grand nombre d'interactions entre les
différentes variables sans avoir à les spécifier explicitement dans un modèle mathématique. C'est une des
multiples raisons pour lesquelles elles performent souvent mieux que d'autres méthodes non linéaires.
<!-- #endregion -->

<!-- #region id="ac779785" -->
# <a id=préparation-des-données>Préparation des données</a>
<!-- #endregion -->

<!-- #region id="b0307464" -->
Nous allons débuter par lire les données, pour cela nous utilisons Pandas qui permet de télécharger automatiquement un jeu de données via un lien URL. Par la suite, nous séparons les étiquettes des données (`y`).
<!-- #endregion -->



In [ ]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]



<!-- #region id="8205cb8e" -->
Par la suite, nous sélectionnons les attributs pouvant servir pour la tâche de régression.
<!-- #endregion -->



In [ ]:
X = pd.DataFrame(
    data,
    columns=[
        "CRIM",
        "ZN",
        "INDUS",
        "CHAS",
        "NOX",
        "RM",
        "AGE",
        "DIS",
        "RAD",
        "TAX",
        "PTRATIO",
        "B",
        "LSTAT",
    ],
)

# Liste des facteurs utilisés
feature_list = list(X.columns)



<!-- #region id="f3c9e153" -->
Nous visualisons ensuite la distribution des prix des maisons.
<!-- #endregion -->



In [ ]:
plt.figure(figsize=(5, 4))
plt.hist(y, 20)
plt.title("Histogramme du prix des maisons")
plt.xlabel("price (k$)")
plt.ylabel("N")
plt.show()



<!-- #region id="df7a36bc" -->
C'est la quantité que l'on désire prédire en fonction des variables $x_{i}$ précédents. Le prix maximum des
maisons est de $50\ 000~\$$.
<!-- #endregion -->

<!-- #region id="822e9f01" -->
Visualisons maintenant nos variables à l'aide de la méthode `.head()` de Pandas.
<!-- #endregion -->



In [ ]:
X.head()



<!-- #region id="618d5126" -->
On voit que les valeurs couvrent différents ordres de grandeur. Il faudra les normaliser pour cette raison. Il faut toutefois mentionner que les arbres décisionnels et les forêts aléatoires ne sont pas affectés par la normalisation. Néanmoins, c'est une bonne habitude de normaliser ses données; autant commencer à prendre cette habitude maintenant!

Débutons donc par séparer nos données en deux ensembles de données selon un ratio $80~\%$ pour l'entraînement et $20~\%$ pour le test.

Les données d'entraînement vont servir à entraînement le modèle de régression. C'est-à-dire, à estimer la valeur du seuil $\tau$ de chaque noeud, dans chaque arbre décisionnel de la forêt. Les données de test vont ensuite servir à mesurer les performances du modèle entraîné à prédire correctement le prix des maisons.

Remplissez les `# À remplir` dans la prochaine cellule.
<!-- #endregion -->



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    # À remplir
    # À remplir
    # À remplir
    random_state=seed
)



<!-- #region id="e2fe1a4c" -->
Par la suite, nous allons normaliser nos données. Remplissez les `# À remplir` pour effectuer une normalisation centrée réduite avec Scikit-learn.
<!-- #endregion -->



In [ ]:
scaler = # À remplir
X_train_s = # À remplir
X_test_s = # À remplir



<!-- #region id="eba93605" -->
# <a id=entraînement-dun-régresseur-par-forêt-aléatoire>Entraînement d'un régresseur par forêt aléatoire</a>
<!-- #endregion -->

<!-- #region id="2eb96959" -->
## <a id=sélection-du-type-de-régresseur>Sélection du type de régresseur</a>
<!-- #endregion -->

<!-- #region id="77febcf6" -->
Définissons maintenant notre régresseur en utilisant la classe [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) de Scikit-learn.
Remplissez le `# À remplir` en utilisant les valeurs de paramètres suivants en utilisant la documentation pour trouver les bons arguments:

- une forêt aléatoire de `100` estimateurs (*estimators*),
- une profondeur des arbres maximales de `6`, et
- un nombre minimal d'exemples (*samples*) de `3` pour diviser une feuille (*internal node*).
<!-- #endregion -->



In [ ]:
reg = # À remplir



<!-- #region id="eba93605" -->
Maintenant, entraînez le régresseur et effectuez les prédictions sur les données d'entraînement et de test.
<!-- #endregion -->



In [ ]:
# À remplir
y_train_pred = # À remplir
y_test_pred = # À remplir



<!-- #region id="e9d9263a" -->
# <a id=affichage-des-résultats>Affichage des résultats</a>
<!-- #endregion -->

<!-- #region id="5bccd345" -->
Calculez maintenant les scores $R^2$ sur l'entraînement et le test.
<!-- #endregion -->



In [ ]:
score = # À remplir
print('Valeur de R2 en entraînement:', round(100 * score, 1), '%')

score = # À remplir
print('Valeur de R2 en test:', round(100 * score, 1), '%')



<!-- #region id="97c90eef" -->
La valeur du $R^2$ correspond à la fraction de la variance des données qui est expliquée par le modèle utilisé.

Le modèle explique environ $95~\%$ de la variabilité des données en entraînement et $87~\%$ de celles en test.
<!-- #endregion -->

<!-- #region id="b0df6bba" -->
Nous allons maintenant afficher des valeurs prédites et réelles des maisons pour l'ensemble de test.
<!-- #endregion -->



In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(y_test, y_test_pred, 'o')
plt.plot(y_test, y_test, '-r', label='Prédiction parfaite')

plt.xlabel('Prix réels (k$)', fontsize=16)
plt.ylabel('Prix prédits (k$)', fontsize=16)
plt.legend()
fig.tight_layout()
plt.show()



<!-- #region id="8c1592f9" -->
La droite rouge correspond à une prédiction parfaite. Mises à part quelques données
aberrantes (*outliers*), il y a une bonne adéquation entre les résultats réels et prédits.
<!-- #endregion -->

<!-- #region id="d844b56c" -->
# <a id=identification-des-variables-les-plus-importantes-déterminant-le-prix-dune-maison>Identification des variables les plus importantes déterminant le prix d'une maison</a>
<!-- #endregion -->

<!-- #region id="c19b36a9" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/inspector-with-magnifying-glass.jpeg"  width="200" />
    <div>
    <font size="0.5">Image Source: http://clipart-library.com/clipart/1416328.htm</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="8f7b5f5e" -->
Il est utile de faire de bonnes prédictions, mais il est aussi intéressant de savoir quelles sont les variables qui affectent le plus le prix des maisons. Les courtiers immobiliers aiment prédire le prix d'une maison qu'ils ont à vendre. Les sociologues, eux, sont curieux de savoir comment les différentes variables socio-économiques affectent l'immobilier, et donc les conditions de vie, dans les différents quartiers d'une ville.

Les variables $x_{i}$ les plus importantes sont les plus utilisées pour prendre
des décisions à travers les arbres décisionnels constituant la forêt.

Le régresseur utilisé, `RandomForestRegressor`, permet de calculer l'importance de chaque variable. Celle-ci prend
une valeur entre 0 et 1, où 0 indique qu'elle n'est jamais utilisée et où 1 indique qu'elle est la seule utilisée
parmi toutes. Dans ce dernier cas, la variable permet de prédire parfaitement la réponse.
La somme des importances sur l'ensemble des variables vaut 1.
<!-- #endregion -->



In [ ]:
# Importance de chaque variable
importances = list(reg.feature_importances_)

# Associe le nom de chaque variable avec son importance
feature_importances = [
    (feature, round(importance, 2))
    for feature, importance in zip(feature_list, importances)
]

# Ordonne les valeurs d'importance en ordre décroissant
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)



<!-- #region id="1d44429a" -->
## <a id=affichage-du-diagramme-dimportance-des-variables-du-jeu-de-données>Affichage du diagramme d'importance des variables du jeu de données</a>
<!-- #endregion -->

<!-- #region id="a2120376" -->
On veut déterminer lesquelles sont les plus importantes et combien il y en a.
<!-- #endregion -->



In [ ]:
indices = np.argsort(importances)[::-1]

plt.style.use('fivethirtyeight')

var = list(range(len(importances)))
plt.bar(var, np.array(importances)[indices.astype(int)], orientation='vertical')
plt.xticks(var, np.array(feature_list)[indices.astype(int)], rotation='vertical')
plt.ylabel('Importance', fontsize=16)
plt.xlabel('Variable', fontsize=16)
plt.title('Importance des variables', fontsize=16)



<!-- #region id="f289b88c" -->
On voit que les variables RM (le nombre moyen de pièces par logement) et LSTAT (le pourcentage des classes défavorisées dans la population) sont de loin les plus importantes. Elles contribuent à $83~\%$ de l'importance totale; elles sont utilisées dans $83~\%$ des décisions des arbres de la forêt.
<!-- #endregion -->

<!-- #region id="66382c4d" -->
## <a id=comment-varie-le-prix-des-maisons-en-fonction-des-deux-variables-les-plus-importantes>Comment varie le prix des maisons en fonction des deux variables les plus importantes?</a>
<!-- #endregion -->

<!-- #region id="2307104b" -->
C'est bien de savoir quelles variables sont les plus déterminantes, c'est encore mieux de savoir comment chacune influence le prix des maisons. C'est ce que nous allons voir avec les deux variables principales.
<!-- #endregion -->

<!-- #region id="3292a1ce" -->
### <a id=sélection-des-variables-dintérêt>Sélection des variables d'intérêt</a>
<!-- #endregion -->



In [ ]:
var_int = ['RM', 'LSTAT']
x = X.loc[:, var_int]
x = pd.DataFrame(data=x, columns=var_int)



<!-- #region id="d6a5ccea" -->
### <a id=affichage-de-leffet-de-chaque-variable-sur-le-prix-des-maisons>Affichage de l'effet de chaque variable sur le prix des maisons</a>
<!-- #endregion -->

<!-- #region id="ac6e62ac" -->
Dans chaque panneau, la droite des moindres carrés est affichée ainsi que l'intervalle de confiance à $95~\%$.
<!-- #endregion -->



In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(10, 5))
index = 0
axs = axs.flatten()
for i, k in enumerate(var_int):
    u = pd.Series(x[k], name="x_var")
    v = pd.Series(y, name="Prix des maisons (k$)")
    sns.regplot(y=v, x=x[k], ax=axs[i])
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)



<!-- #region id="6a003dc0" -->
On voit que le prix d'une maison augmente avec le nombre de chambres (RM) et diminue avec le
le pourcentage des classes défavorisées dans la population (LSAT). Une conclusion similaire
pourrait être tirée de bases de données contemporaines pour un très grand nombre de villes dans le monde.
<!-- #endregion -->

<!-- #region id="65bc2627" -->
Quelles seraient l'effet des performances (la valeur du $R^2$) si la forêt aléatoire n'utilisait que
les deux variables les plus importantes, soit `RM` et `LSAT`? 
<!-- #endregion -->

<!-- #region id="4a2875f3" -->
Testez cet effet en effectuant l'entraînement de votre redresseur à nouveau, mais en utilisant cette fois uniquement les variables `RM` et `LSAT` et en utilisant le régresseur ayant les caractéristiques suivantes:

- une forêt aléatoire de `100` estimateurs (*estimators*),
- une profondeur des arbres maximales de `4`, et
- un nombre minimal d'exemples (*samples*) de `5` pour diviser une feuille (*internal node*).
<!-- #endregion -->



In [ ]:
# 1. Extraire les deux variables seulement des données
# 2. Refaire la séparation des données
# 3. Normalisation
# 4. Entraînement du régresseur
# 5. Prédiction et évaluation



<!-- #region id="dd080c7f" -->
Affichez à nouveau les valeurs prédites et réelles du prix des maisons pour l'ensemble de test. En quoi la figure
est-elle différente de la version précédente?
<!-- #endregion -->

